In [3]:
import sqlite3
conn = sqlite3.connect("cust_database.db")

create="""
CREATE TABLE IF NOT EXISTS customer(
c_ID INTEGER PRIMARY KEY AUTOINCREMENT,
name,
phone int,
address)
"""

cursor=conn.cursor()
cursor.execute(create)


In [4]:
create_products = """
CREATE TABLE IF NOT EXISTS products (
    p_ID INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    price REAL,
    year_produced INTEGER,
    description TEXT
)
"""
cursor.execute(create_products)
conn.commit()


In [71]:

class customer:
    def __init__(self):
        pass
        
    def add_acount(self, name, phone, address):
        add = "INSERT INTO customer (name, phone, address) VALUES (?, ?, ?)"
        cursor.execute(add, (name, phone, address))
        conn.commit()
        customer_id = cursor.lastrowid
        table_name = f"basket_{customer_id}"

        # Create personal basket table
        create_basket = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            basket_id INTEGER PRIMARY KEY AUTOINCREMENT,
            product_id INTEGER,
            quantity INTEGER,
            FOREIGN KEY (product_id) REFERENCES products(p_ID)
        )
        """
        cursor.execute(create_basket)
        conn.commit()
        print("account created")


    def customer_list(self):
        lis="SELECT * FROM customer"
        a=cursor.execute(lis)
        print(a.fetchall())

    def delete_account(self,id):
        delete = "DELETE FROM customer WHERE c_ID = ?"
        cursor.execute(delete,(str(id)))
        conn.commit()

    def whats_my_ID(self,name,phone):
        t="SELECT c_ID FROM customer WHERE name = ? AND phone = ?"
        cursor.execute(t,(name,phone))
        iD=cursor.fetchone()
        print(f"your ID is {iD} . enjoy :)")





In [72]:
class products:
    def __init__(self):
        pass

    def add_product(self,name,price,year_produced,description):
        add = "INSERT INTO products (name, price, year_produced , description) VALUES (?,?,?,?)"
        cursor.execute(add ,(name, price, year_produced, description))
        conn.commit()

    def whats_product_iD(self,name,year_produced):
        t="SELECT p_ID FROM products WHERE name = ? AND year_produced = ?"
        cursor.execute(t,(name,year_produced))
        iD=cursor.fetchone()
        print(f"the product ID is {iD} .")

    def delete_product(self,id):
        delete = "DELETE FROM products WHERE p_ID = ?"
        cursor.execute(delete,(str(id)))
        conn.commit()

    def products_list(self):
        lis="SELECT * FROM products"
        a=cursor.execute(lis)
        print(a.fetchall())

    

In [73]:
class shop(products, customer):

    def add_to_basket(self, customer_id, product_id, quantity=1):
        table = f"basket_{customer_id}"
        check = f"SELECT quantity FROM {table} WHERE product_id = ?"
        cursor.execute(check, (product_id,))
        result = cursor.fetchone()

        if result==True:
            new_qty = result[0] + quantity
            update = f"UPDATE {table} SET quantity = ? WHERE product_id = ?"
            cursor.execute(update, (new_qty, product_id))
        else:
            insert = f"INSERT INTO {table} (product_id, quantity) VALUES (?, ?)"
            cursor.execute(insert, (product_id, quantity))

        conn.commit()
        print(f"Added {quantity} of product {product_id} to basket")

    def view_basket(self, customer_id):
        table = f"basket_{customer_id}"
        query = f"""
        SELECT p.p_ID, p.name, p.price, b.quantity, (p.price * b.quantity) as total_price
        FROM {table} b
        JOIN products p ON b.product_id = p.p_ID
        """
        cursor.execute(query)
        items = cursor.fetchall()

        if items:
            total = 0
            for item in items:
                print(f"ID: {item[0]}, Name: {item[1]}, Price: {item[2]}, Qty: {item[3]}, Total: {item[4]}")
                total += item[4]
            print(f" Total: {total}")
        else:
            print(f"empty")

    def remove_from_basket(self, customer_id, product_id, quantity=1):
        table = f"basket_{customer_id}"
        check = f"SELECT quantity FROM {table} WHERE product_id = ?"
        cursor.execute(check, (product_id,))
        result = cursor.fetchone()

        if result:
            current_qty = result[0]
            if current_qty > quantity:
                update = f"UPDATE {table} SET quantity = ? WHERE product_id = ?"
                cursor.execute(update, (current_qty - quantity, product_id))
                print(f"Removed {quantity} of product {product_id}")
            else:
                delete = f"DELETE FROM {table} WHERE product_id = ?"
                cursor.execute(delete, (product_id,))
                print(f"Removed product {product_id}")
            conn.commit()
        else:
            print("not in basket.")

    def checkout(self, customer_id):
        table = f"basket_{customer_id}"

        delete = f"DELETE FROM {table}"
        cursor.execute(delete)
        conn.commit()
        print("complete")



In [85]:
s=shop()

### customer related:

In [9]:
s.add_acount("kasra","0912","tehran")

account created


In [10]:
s.whats_my_ID("kasra","0912")

your ID is (1,) . enjoy :)


In [11]:
s.add_acount("ali","0910","karaj")

account created


In [31]:
s.customer_list()

[(1, 'kasra', 912, 'tehran'), (2, 'ali', 910, 'karaj')]


In [75]:
s.delete_account("2")

In [76]:
s.customer_list()

[(1, 'kasra', 912, 'tehran')]


In [94]:
s.add_acount("ali","0910","karaj")

account created


### product related:

In [77]:
s.add_product("laptop","12000","2020","gaming laptop")

In [78]:
s.products_list()

[(1, 'laptop', 12000.0, 2020, 'gaming laptop')]


In [79]:
s.add_product("TV","32000","2022","24 inch")

In [80]:
s.products_list()

[(1, 'laptop', 12000.0, 2020, 'gaming laptop'), (2, 'TV', 32000.0, 2022, '24 inch')]


In [81]:
s.whats_product_iD("TV","2022")

the product ID is (2,) .


In [82]:
s.delete_product(2)

In [83]:
s.products_list()

[(1, 'laptop', 12000.0, 2020, 'gaming laptop')]


In [99]:
s.add_product("TV","32000","2022","24 inch")

### basket related:

In [108]:
s.add_to_basket(1,1,2)

Added 2 of product 1 to basket


i ran this part twice:

In [109]:
s.view_basket(1)

ID: 1, Name: laptop, Price: 12000.0, Qty: 2, Total: 24000.0
ID: 1, Name: laptop, Price: 12000.0, Qty: 2, Total: 24000.0
 Total: 48000.0


In [112]:
s.remove_from_basket(1,1,1)

Removed 1 of product 1


In [113]:
s.view_basket(1)

ID: 1, Name: laptop, Price: 12000.0, Qty: 1, Total: 12000.0
ID: 1, Name: laptop, Price: 12000.0, Qty: 1, Total: 12000.0
 Total: 24000.0


In [114]:
s.checkout(1)

complete


In [116]:
s.view_basket(1)

empty
